In [2]:
import numpy as np
import pandas as pd
import random
from copy import deepcopy
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import os
np.random.seed(7)

In [15]:
# create sequence dataset
# def create_dataset(dataset, label, seq_len=50):
#     X, Y = [], []
#     for i in range(len(dataset) - seq_len + 1):
#         a = dataset[i:(i + seq_len)]
#         X.append(a)
#         Y.append(label)
#     return np.array(X), np.array(Y)
def create_dataset(dataset, label, seq_len=100):
    X, Y = [], []
    num = int(len(dataset) / seq_len)
    for i in range(num):
        a = dataset[i * num:(i * num + seq_len)]
        X.append(a)
        Y.append(label)
    return np.array(X), np.array(Y)

In [ ]:
# generator
# def generate_batch_data(batch_size, seq_len):
#     for _, _, files in os.walk('malicious_train'):
#         for filename in files:
#             ip, label = filename.split('_')
#             df = pd.read_csv(f"malicious_train/{filename}", header=None)
#             dataset = []            

In [3]:
# read csv data
df_irc = pd.read_csv('malicious_train/147.32.84.180.csv', header=None)
df_http = pd.read_csv('malicious_train/147.32.84.140.csv', header=None)
df_p2p = pd.read_csv('malicious_train/192.168.4.118.csv', header=None)

In [4]:
# dataset_benign, dataset_malicious = df_benign.values, df_malicious.values
# dataset_benign.shape, df_malicious.shape
dataset_irc, dataset_http, dataset_p2p = df_irc.values, df_http.values, df_p2p.values
dataset_irc.shape, dataset_http.shape, dataset_p2p.shape

((46713, 1500), (6873, 1500), (11370, 1500))

In [38]:
Y = ['IRC', 'HTTP', 'P2P']
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
labels = np_utils.to_categorical(encoded_Y)
labels

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [40]:
# read csv
flag = 0
dir_list = os.listdir('test')
dir_list.remove('.DS_Store')
print(dir_list)
for folder in dir_list:
    print(folder)
    for _, _, files in os.walk(f"test/{folder}"):
        for filename in files:
            if filename == '.DS_Store':
                continue
            print(filename)
            df = pd.read_csv(f"test/{folder}/{filename}", header=None)
            dataset = df.values
            X, Y = create_dataset(dataset, labels[dir_list.index(folder)])
            print("X shape: " + str(X.shape))
            if flag == 0:
                X_train, Y_train = X, Y
                flag = 1
            else:
                X_train, Y_train = np.concatenate((X_train, X)), np.concatenate((Y_train, Y))
            print(X_train.shape)
        

['IRC', 'HTTP']
IRC
172.16.253.240.csv
X shape: (37, 100, 500)
(37, 100, 500)
172.16.253.130.csv
X shape: (42, 100, 500)
(79, 100, 500)
172.16.253.131.csv
X shape: (31, 100, 500)
(110, 100, 500)
147.32.84.150.csv
X shape: (45, 100, 500)
(155, 100, 500)
147.32.84.180.csv
X shape: (467,)


ValueError: all the input arrays must have same number of dimensions

In [6]:
# create and concatenate benign and malicious dataset
# X_train, Y_train = create_dataset(dataset_benign_train, 0)
# X_test, Y_test = create_dataset(dataset_benign_test, 0)
# X_tmp_train, Y_tmp_train = create_dataset(dataset_malicious_train, 1)
# X_tmp_test, Y_tmp_test = create_dataset(dataset_malicious_test, 1)
# X_train, Y_train = np.concatenate((X_train, X_tmp_train)), np.concatenate((Y_train, Y_tmp_train))
# X_test, Y_test = np.concatenate((X_test, X_tmp_test)), np.concatenate((Y_test, Y_tmp_test))
# X_train.shape, X_test.shape

X_irc, Y_irc = create_dataset(dataset_irc, Y[0])
X_http, Y_http = create_dataset(dataset_http, Y[1])
X_p2p, Y_p2p = create_dataset(dataset_p2p, Y[2])
X_train, Y_train = np.concatenate((X_irc, X_http, X_p2p)), np.concatenate((Y_irc, Y_http, Y_p2p))
X_train.shape, Y_train.shape

((64809, 50, 1500), (64809, 3))

In [8]:
# create model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 48, 64)            288064    
_________________________________________________________________
dropout_3 (Dropout)          (None, 48, 64)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 300,579
Trainable params: 300,579
Non-trainable params: 0
_________________________________________________________________


In [9]:
# fit network
model.fit(X_train, Y_train, epochs=1, batch_size=100, verbose=True)

W0722 19:18:20.054501 4560844224 deprecation.py:323] From /Users/lee/venv3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
64809/64809 [==============================] - 465s 7ms/step - loss: 0.4137 - acc: 0.8407


In [10]:
df_eval = pd.read_csv('malicious_test/147.32.84.160.csv')

In [ ]:
dataset_eval = df_eval.values
X_eval, Y_eval = create_dataset(dataset_eval, Y[1])
X_eval.shape, Y_eval.shape

In [30]:
# evaluation
_, accur = model.evaluate(X_eval, Y_eval, batch_size=100)
accur

1600/1600 [==============================] - 1s 651us/step


0.4243750001769513